In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob
from tqdm import tqdm
import h5py
from PIL import Image
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
batch_size = 64
train_dir = os.path.join("resized_images", "train")

datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory(batch_size=batch_size,
                                         directory=train_dir,
                                         shuffle=True,
                                         class_mode='categorical')
# Confirm iterator works
batchX, batchY = train_data.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 105392 images belonging to 42 classes.
Batch shape=(64, 256, 256, 3), min=0.000, max=255.000


In [3]:
# for label in tqdm(range(0,1)):
#     np_images = []
#     np_labels = []
#     images = glob.glob(f"resized_images/train/{label:02d}/*.jpg")

#     for i in range(len(images)):
#         image = Image.open(images[i])
#         np_image = np.asarray(image)
#         np_images.append(np_image/255)
#         np_labels.append(label)

#     data_x = np.asarray(np_images)
#     data_y = np.asarray(np_labels)
#     hf = h5py.File(f'pickles/data_{label}.h5', 'w')
#     hf.create_dataset(f'x', data=data_x,  compression='gzip', compression_opts=9)
#     hf.create_dataset(f'y', data=data_y)
#     hf.close()


0%|          | 0/1 [00:13<?, ?it/s]


TypeError: 'compression' is an invalid keyword argument for this function

In [42]:
def get_model():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, padding='same', activation = 'relu', input_shape = (224, 224, 3))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size = 2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(filters = 16, kernel_size = 2, padding = 'same', activation = 'relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = 2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(42, activation = 'softmax'))

    # Take a look at the model summary
    model.summary()
    return model

In [43]:
model = get_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 32)      416       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 16)      2064      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)            

In [44]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [45]:
model.fit(
    x=data_x,
    y=data_y,
    epochs=10)

Epoch 1/10
3/3 [==============================] - 2s 544ms/step - loss: 3.7646 - accuracy: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 2s 533ms/step - loss: 3.7649 - accuracy: 0.0119
Epoch 3/10
3/3 [==============================] - 1s 495ms/step - loss: 3.7537 - accuracy: 0.0238
Epoch 4/10
3/3 [==============================] - 2s 530ms/step - loss: 3.7538 - accuracy: 0.0238
Epoch 5/10
3/3 [==============================] - 2s 517ms/step - loss: 3.7540 - accuracy: 0.0238
Epoch 6/10
3/3 [==============================] - 2s 507ms/step - loss: 3.7539 - accuracy: 0.0238
Epoch 7/10
3/3 [==============================] - 2s 548ms/step - loss: 3.7540 - accuracy: 0.0238
Epoch 8/10
3/3 [==============================] - 2s 509ms/step - loss: 3.7540 - accuracy: 0.0238
Epoch 9/10
3/3 [==============================] - 1s 455ms/step - loss: 3.7540 - accuracy: 0.0238
Epoch 10/10
3/3 [==============================] - 1s 483ms/step - loss: 3.7540 - accuracy: 0.0238


In [47]:
model.evaluate(
    x=data_x,
    y=data_y
)

3/3 [==============================] - 1s 169ms/step - loss: 3.7540 - accuracy: 0.0238


[3.7539570331573486, 0.02380952425301075]